In [ ]:
import numpy as np
import os
import random
from tqdm.notebook import tqdm
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchvision import transforms
import zipfile

In [ ]:
data_dir = '/kaggle/datasets/'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
with zipfile.ZipFile('../input/lux-ai-datasets/datasets.zip') as z:
    z.extractall(data_dir)

In [ ]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True


seed = 42
seed_everything(seed)

In [ ]:
class LuxAugment:
    def __init__(self):
        self.one_hot = np.eye(6)
    
    def __call__(self, sample):
        state, action = sample
        # Flip vertically
        if random.random() < 0.5:
            state = np.flip(state, axis=1).copy()
            action = self.one_hot[np.flip(action, axis=0)][:,:,[0,1,2,4,3,5]].argmax(-1)

        # Flip horizontally
        if random.random() < 0.5:
            state = np.flip(state, axis=2).copy()
            action = self.one_hot[np.flip(action, axis=1)][:,:,[0,2,1,3,4,5]].argmax(-1)

        # Rotate 90 degrees
        if random.random() < 0.5:
            state = np.rot90(state, axes=(1, 2)).copy()
            action = self.one_hot[np.rot90(action, axes=(0, 1))][:,:,[0,3,4,2,1,5]].argmax(-1)
        
        return state, action


class LuxDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.samples = os.listdir(data_dir)
        self.transform = transform
        
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        data_path = os.path.join(self.data_dir, self.samples[idx])
        data = np.load(data_path)
        sample = data['state'], data['action']
        if self.transform is not None:
            sample = self.transform(sample)
        
        return sample

In [ ]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, n_classes):
        super(DiceLoss, self).__init__()
        self.n_classes = n_classes

    def _one_hot_encoder(self, input_tensor):
        tensor_list = []
        for i in range(self.n_classes):
            temp_prob = input_tensor == i
            tensor_list.append(temp_prob.unsqueeze(1))
        output_tensor = torch.cat(tensor_list, dim=1)
        return output_tensor.float()

    def _dice_loss(self, score, target):
        target = target.float()
        smooth = 1e-5
        intersect = torch.sum(score * target)
        y_sum = torch.sum(target * target)
        z_sum = torch.sum(score * score)
        loss = (2 * intersect + smooth) / (z_sum + y_sum + smooth)
        loss = 1 - loss
        return loss

    def forward(self, inputs, target):
        inputs = torch.softmax(inputs, dim=1)
        target = self._one_hot_encoder(target)
        assert inputs.size() == target.size(), 'predict {} & target {} shape do not match'.format(inputs.size(), target.size())
        class_wise_dice = []
        loss = 0.0
        for i in range(0, self.n_classes):
            dice = self._dice_loss(inputs[:, i], target[:, i])
            class_wise_dice.append(1.0 - dice.item())
            loss += dice
        return loss / self.n_classes

In [ ]:
def train_model(model, dataloader, criterion, optimizer, scheduler, num_epochs):
    model.train()
    model.cuda()
    
    for epoch in range(num_epochs):                
        epoch_loss = 0.0
        
        for item in tqdm(dataloader, leave=False):
            states = item[0].cuda().float()
            actions = item[1].cuda().long()

            optimizer.zero_grad()

            policy = model(states)
            loss = criterion(policy, actions)

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * len(policy)

        epoch_loss = epoch_loss / len(dataloader.dataset)
        print(f'Epoch {epoch + 1}/{num_epochs} | Loss: {epoch_loss:.4f} | LR: {scheduler.get_last_lr()[0]:.6f}')
        scheduler.step()
    
    model.eval()
    model.cpu()
    traced = torch.jit.trace(model, torch.rand(1, model.n_channels, 32, 32))
    traced.save('model.pth')

In [ ]:
model = UNet(n_channels=18, n_classes=6)
dataloader = DataLoader(
    LuxDataset(data_dir, transforms.Compose([LuxAugment()])),
    batch_size=128, 
    shuffle=True, 
    num_workers=2
)
criterion = DiceLoss(model.n_classes)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=50, eta_min=1e-6)

In [ ]:
train_model(model, dataloader, criterion, optimizer, scheduler, num_epochs=50)